In [16]:
# Adapted from Coldstart Coder on Medium (2024)
# https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f

from pyvirtualdisplay import Display

# Do not move these, it will break
display = Display(visible=0, size=(1400, 900))
display.start()

from typing import Callable
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import io
import base64
from IPython import display
from IPython.display import HTML

import stormvogel.result
import stormvogel.model


def embed_video(video_file):
    # open and read the raw data from the video
    video_data = io.open(video_file, 'r+b').read()
    # now we have to encode the data into base64 to work
    # with the virtual display
    encoded_data = base64.b64encode(video_data)
    # now we use the display.display function to take some html
    # and the encoded data and embed the html into the notebook!
    display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded_data.decode('ascii'))))

def create_video(env, location, choose_action: Callable | None = None) -> str:
    """Create a video of the current environment using the choose_action function to choose an action.
    Args:
        env: The gymnasium env
        location: Path where the video should be stored
        choose_action: function that chooses the action. Choose a random action if left as None.

    Returns: (relative) path to video.
    """
    env = RecordVideo(env, location)
    state, inf = env.reset()
    while True:
        # render the frame, this will save it to the video file
        env.render()
        # pick a random action
        if choose_action is None:
            action = env.action_space.sample()
        else:
            action = choice_function(state)
        # run the action
        state, reward, terminated, truncated, info = env.step(action)
        # if the simulation has ended break the loop
        if terminated:
            break

    env.close()
    return location + "/rl-video-episode-0.mp4"

dir_map = { "left":0, "down":1, "right":2, "up":3, "pickup":4, "dropoff":5 }
inv_dir_map = {v: k for k, v in dir_map.items()}

def create_video_scheduler(
    env, location, 
    scheduler:  stormvogel.result.Scheduler
                | Callable[stormvogel.model.State, stormvogel.model.Action]
                | None = None) -> str:
    """Create a video of the current environment using the scheduler to choose an action.
    Args:
        env: The gymnasium env
        location: Path where the video should be stored
        scheduler: A function that chooses the action, or None

    Returns: (relative) path to video.
    """
    if isinstance(scheduler, stormvogel.result.Scheduler):
        def choose_action(s):
            return dir_map[scheduler.get_choice_of_state(s).labels[0]]
    elif callable(scheduler):
        def choose_action(s):
            return dir_map(scheduler(s))
    else:
        choose_action = None
    return create_video(env, location, choose_action)
    
    

In [12]:
env = gym.make("FrozenLake-v1", render_mode="rgb_array")
#embed_video(create_video(env, './video'))